In [1]:
import sys
import time
import math
import json
import operator
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

In [2]:
class Node:
    
    node_count = 0
    
    def __init__(self, attribute_name, is_continuous, threshold, total_positive, total_negative, is_leaf, label, branch):
        self.attribute_name = attribute_name
        self.is_continuous = is_continuous
        self.threshold = threshold
        self.total_positive = total_positive
        self.total_negative = total_negative
        self.is_leaf = is_leaf
        self.label = label
        self.branch = branch
        Node.node_count += 1
        
    def get_total_nodes(self):
        return self.node_count

In [3]:
def read_temp(path):
    column_names = ['days', 'outlook', 'temperature', 'humidity', 'wind', 'decision']
    data = pd.read_csv(path, names = column_names)
    data = data.drop(['days'], axis = 1)
    return data
    

In [4]:
def read_data(train_data_path, dev_data_path, test_data_path):
    column_names = ['age', 'workclass', 'education', 'marital-status', 'occupation', 'race', 'sex', 'hours', 'country', 'income']
    train_data = pd.read_csv(train_data_path, names = column_names)
    dev_data = pd.read_csv(dev_data_path, names = column_names)
    test_data = pd.read_csv(test_data_path, names = column_names)
    
    return (train_data, dev_data, test_data)

In [5]:
def get_initial_values(data):
    
    attributes = data.columns.values
    index = np.argwhere(attributes == attributes[-1])
    attributes = np.delete(attributes, index)
    
    labels = np.unique(data.iloc[:, -1])
    
    positive_label = labels[0]
    negative_label = labels[1]
    
    return attributes, positive_label, negative_label

In [24]:
class DecisionTree:
    

    def __init__(self, positive_label_val, negative_label_val):
        self.positive_label_val = positive_label_val
        self.negative_label_val = negative_label_val
    
    def build_tree(self, data, attributes):
        
        # Create a node
        node = Node(None, False, None, None, None, None, False, None)

        # Get positive and negative label
        values, count = np.unique(data.iloc[:, -1], return_counts = True)
        val_count = dict(zip(values, count))
        
        positive_label = 0
        negative_label = 0
        
        if(len(val_count) == 2):
            positive_label = list(val_count)[0]
            negative_label = list(val_count)[1]
            node.total_positive = val_count[positive_label]
            node.total_negative = val_count[negative_label]
        else:
            if(self.positive_label_val == list(val_count.keys())[0]):
                positive_label = list(val_count)[0]
                negative_label = 0 
                node.total_positive = val_count[positive_label]
                node.total_negative = 0
            else:
                positive_label = 0
                negative_label = list(val_count)[0]
                node.total_positive = 0
                node.total_negative = val_count[negative_label]

        
            
        if(node.total_negative == 0):
            node.is_leaf = True
            node.label = positive_label
            return node

        if(node.total_positive == 0):
            node.is_leaf = True
            node.label = negative_label
            return node

        if(len(attributes) == 0):
            node.is_leaf = True
            if(node.total_positive > node.total_negative):
                node.label = positive_label
            else:
                node.label = negative_label
            return node

        else:

            info_gains = {}
            thresholds = {}

            for attribute in attributes:
                if(np.issubdtype(train_data[attribute].dtype.name, np.integer)):
                    thresholds[attribute], info_gains[attribute] = self.calculate_threshold(data[attribute], data.iloc[:, -1])
                else:
                    info_gains[attribute] = self.information_gain(data[attribute], data.iloc[:, -1])

            # Attribute with maximum gain
            max_gain_attr = max(info_gains.items(), key=operator.itemgetter(1))[0]

            # If continuous attribute, set threshold value
            if(max_gain_attr in list(thresholds.keys())):
                node.threshold = thresholds[max_gain_attr]
                node.is_continuous = True

            node.attribute_name = max_gain_attr
            node.branch = {}
            
            # Check if the best attribute is continuous or categorical
            if(node.is_continuous):
                for value in ['True', 'False']:
                    if(value == 'True'):
                        data[max_gain_attr] = np.where(data[max_gain_attr] < node.threshold, 'True', 'False')
                    node.branch[value] = None
                    subset = data[data[max_gain_attr] == value]
                    if(subset.shape[0] == 0):
                        node.branch[value] = Node(None, True, None, None, None, None, True, None)
                        c, v = np.unique(data.iloc[:,-1], return_counts = True)
                        c_v = dict(zip(c, v))
                        key = max(c_v.items(), key=operator.itemgetter(1))[0]
                        node.branch[value].label = key
                    else:
                        index = np.argwhere(attributes == max_gain_attr)
                        attributes = np.delete(attributes, index)
                        node.branch[value] = self.build_tree(subset, attributes)
                
            else:
                # For each unique value from attribute column
                for value in np.unique(data[max_gain_attr]):
                    node.branch[value] = None
                    subset = data[data[max_gain_attr] == value]
                    if(subset.shape[0] == 0):
                        node.branch[value] = Node(None, False, None, None, None, None, True, None)
                        c, v = np.unique(data.iloc[:,-1], return_counts = True)
                        c_v = dict(zip(c, v))
                        key = max(c_v.items(), key=operator.itemgetter(1))[0]
                        node.branch[value].label = key
                    else:
                        index = np.argwhere(attributes == max_gain_attr)
                        attributes = np.delete(attributes, index)
                        node.branch[value] = self.build_tree(subset, attributes)

        return node
    
    def calculate_threshold(self, data, label):
        data = data.values
        label = label.values
        indexes = data.argsort()
        data = np.flip(data[indexes[::-1]])
        label = np.flip(label[indexes[::-1]])
        label_df = pd.DataFrame(label)
        candidate_threshold = []
        info_gains = []

        for i in range(data.size - 1):
            threshold = data[i] + (data[i + 1] - data[i]) / 2
            if threshold not in candidate_threshold:
                candidate_threshold.append(threshold)
            else:
                continue

        for threshold in candidate_threshold:
            values = data < threshold
            values_df = pd.DataFrame(values)
            gain = self.information_gain(values_df, label_df)
            info_gains.append(gain)

        m = max(info_gains)
        max_posn = [i for i, j in enumerate(info_gains) if j == m]
        candidate_threshold = np.array(candidate_threshold)
        threshold = candidate_threshold[max_posn][0]
        
        return threshold, m
    
    def information_gain(self, data, label):
        attributes, count = np.unique(data, return_counts = True)
        attribute_count = dict(zip(attributes, count))
        label = label.values
        data = data.values
        entropies = []
        for attribute in attribute_count:
            index = np.where(data == attribute)
            op_cls = np.take(label, index)[0]
            ops, total = np.unique(op_cls, return_counts = True)
            ops_total = dict(zip(ops, total))
            entropy = 0
            try:
                entropy = self.calculate_entropy(ops_total[list(ops_total)[0]], ops_total[list(ops_total)[1]])
            except:
                entropy = self.calculate_entropy(ops_total[list(ops_total)[0]])
            entropy = entropy * (attribute_count[attribute] / len(label))
            entropies.append(entropy)
        ops, total = np.unique(label, return_counts = True)
        ops_total = dict(zip(ops, total))
        
        entropy = self.calculate_entropy(ops_total[list(ops_total)[0]], ops_total[list(ops_total)[1]])
        information = entropy - np.sum(entropies)
        return information
    
    def calculate_entropy(self, positive, negative = 0):
        total = positive + negative
        if negative == 0:
            return - (positive/total) * math.log((positive/total), 2)
        else:
            return - (positive/total) * math.log((positive/total), 2) - (negative/total) * math.log((negative/total), 2)
        
    
    def score(self, data, node):
        labels = data.iloc[:, -1].values
        data = data.values
        
        print(labels)
        print(data)
    

In [25]:
if __name__ == '__main__':
    train_data_path = 'income-data/income.train.txt'
    dev_data_path = 'income-data/income.dev.txt'
    test_data_path = 'income-data/income.test.txt'
    temp_data_path = 'income-data/data.csv'
    
    (train_data, dev_data, test_data) = read_data(train_data_path, dev_data_path, test_data_path)
    
    #train_data = read_temp(temp_data_path)
    
    (attributes, positive_lab, negative_lab) = get_initial_values(train_data)
    
    # Create an instance of Decision Tree
    dt = DecisionTree(positive_lab, negative_lab)
    
    # Build the tree
    root = dt.build_tree(train_data, attributes)
    
    
    

In [50]:
# Get training accuracy
train_accuracy = score(train_data, root)
#dev_acuracy = score(dev_data, root)
#test_accuracy = score(test_data, root)

 Married-civ-spouse
 Doctorate
 United-States
 Private
 Prof-specialty
40
60
 Male
 White
 Married-civ-spouse
 Bachelors
 Tech-support
 Private
50
 United-States
24
 Male
 White
 Divorced
 Some-college
 Tech-support
 Private
 Married-civ-spouse
 HS-grad
 Protective-serv
 Local-gov
49
40
 White
 Male
 United-States
 Never-married
 HS-grad
 Local-gov
 Divorced
 Some-college
 Craft-repair
39
 Male
 Private
40
 White
 United-States
 Divorced
 Some-college
 Adm-clerical
47
 Female
 Local-gov
 Married-civ-spouse
 Some-college
 Sales
51
 United-States
 Private
40
 White
 Male
 Married-civ-spouse
 Some-college
 Tech-support
47
 Private
 United-States
45
 White
 Male
 Married-civ-spouse
 7th-8th
 Machine-op-inspct
54
 Never-married
 Some-college
 Adm-clerical
40
38
 State-gov
 Divorced
 7th-8th
 Never-married
 Bachelors
22
 Private
 Prof-specialty
 White
15
 Never-married
 Some-college
 Other-service
 Female
 Divorced
 HS-grad
 Transport-moving
35
45
 Male
 Private
 White
 United-States
 Never-

KeyError: ' Female'

In [49]:
def score(data, node):
    root = node
    y_hat = []
    y = data.iloc[:, -1].values
    x = data.drop(data.columns.values[-1], axis = 1)
    
    for key, value in x.iterrows():
        while(node.is_leaf != True):
            val = x[node.attribute_name][key]
            print(val)
            if(np.issubdtype(x[node.attribute_name].dtype.name, np.integer)):
                if(val < node.threshold):
                    node = node.branch['True']
                else:
                    node = node.branch['False']
            else:
                node = node.branch[val]     
        y_hat.append(node.label)
        node = root
        
    y_hat = np.asarray(y_hat)
    accuracy = calculate_accuracy(y_hat, y)
    print(accuracy)
    

In [28]:
def calculate_accuracy(y_hat, y):
    count = np.equal(y_hat, y)
    value, count = np.unique(count, return_counts = True)
    val_count = dict(zip(value, count))
        
    accuracy = 1 - (val_count[False] / y_hat.shape[0])
        
    return accuracy